In [ ]:
# Env
!pip install mrcnn
!pip install tensorflow==1.14.0
!pip install tensorflow==1.3.0
!pip install keras==2.0.8

In [ ]:
import mrcnn
import numpy as np
import colorsys
import argparse
import imutils
import random
import cv2
import os

from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
from matplotlib import pyplot
from matplotlib.patches import Rectangle


%matplotlib inline

from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset

from mrcnn.model import MaskRCNN

In [2]:
class myMaskRCNNConfig(Config):
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes (we would normally add +1 for the background)
     # kangaroo + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 32
    
    # Learning rate
    LEARNING_RATE=0.006
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10

In [3]:
config= myMaskRCNNConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.006
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [4]:
# initialize the Mask R-CNN model for inference 
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="inference", config=config, model_dir='./')

Loading Mask R-CNN model...


In [ ]:
model.keras_model.summary()

In [8]:
from numpy import save,load

PATH            = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/'
TRAIN_PATH      = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train/'
TRAIN_MASK_PATH = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train_mask/'
TRAIN_ELA_PATH  = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train_ela/'
TEST_PATH       = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/test/'
TEST_ELA_PATH   = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/test_ela/'
CKPT_PATH       = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/model/'

X_train = load(PATH+'npy/X_train.npy')
y_train  = load(PATH+'npy/Y_train.npy')
X_valid = load(PATH+'npy/X_val1.npy')
y_valid  = load(PATH+'npy/Y_val.npy')
X_test1 = load(PATH+'npy/X_test1.npy')